# Lesson 09 — Google ADK: A2A Client

> **Prerequisite:** Run all cells in `server.ipynb` first — the server must be running on **port 10091** before continuing.

This notebook shows how an **A2A client** discovers and queries the ADK-powered agent using the `a2a-sdk` `ClientFactory`.

```mermaid
sequenceDiagram
    participant C as client.ipynb
    participant S as server.ipynb :10091
    participant A as PolicyQAAgent (ADK + Phi-4)
    C->>S: GET /.well-known/agent-card.json
    S-->>C: AgentCard (name · skills · version)
    C->>S: POST / (message/send)
    S->>A: LlmAgent.run(question)
    A-->>S: Task with history
    S-->>C: (Task, Event) tuple
```


In [ ]:
# ── Imports ───────────────────────────────────────────────────────
# Note: no asyncio.run() needed — Jupyter supports top-level `await`
import httpx

from a2a.client import ClientFactory
from a2a.client.client import ClientConfig
from a2a.types import Message, Part, Role, TextPart

SERVER_URL = "http://localhost:10091"


def _get_client_config() -> ClientConfig:
    """Fresh ClientConfig per call — avoids closed event-loop errors."""
    return ClientConfig(
        httpx_client=httpx.AsyncClient(timeout=120.0),
        streaming=False,
    )


print("Imports ready.")
print(f"Server URL: {SERVER_URL}")

In [ ]:
# ── Step 1: Discover the agent card ──────────────────────────────


async def discover_agent() -> dict:
    client = await ClientFactory.connect(
        agent=SERVER_URL,
        client_config=_get_client_config(),
    )
    card = await client.get_card()
    return {
        "name": card.name,
        "description": card.description,
        "version": card.version,
        "skills": [s.name for s in (card.skills or [])],
    }


info = await discover_agent()
print("✓ Discovered agent")
print(f"  Name        : {info['name']}")
print(f"  Description : {info['description']}")
print(f"  Version     : {info['version']}")
print(f"  Skills      : {info['skills']}")

In [ ]:
# ── Step 2: Send a question ───────────────────────────────────────
#
# ADK returns (Task, Event) tuples via send_message().
# Walk task.history in reverse to find the last agent message,
# skipping internal thought parts (adk_thought=True in metadata).


def _extract_text(item: object) -> str:
    """Extract the agent's text from a send_message item."""
    if isinstance(item, Message):
        texts = [p.root.text for p in item.parts if hasattr(p.root, "text")]
        return "\n".join(texts) if texts else "(no text in response)"

    if isinstance(item, tuple):
        task = item[0]
        if hasattr(task, "history") and task.history:
            for msg in reversed(task.history):
                if getattr(msg, "role", None) != Role.agent:
                    continue
                texts = [
                    p.root.text
                    for p in msg.parts
                    if hasattr(p, "root") and hasattr(p.root, "text")
                    # skip ADK internal thought parts
                    and not (getattr(p.root, "metadata", None) or {}).get("adk_thought")
                ]
                if texts:
                    return "\n".join(texts)

    return f"(unexpected response type: {type(item).__name__})"


async def ask(question: str) -> str:
    client = await ClientFactory.connect(
        agent=SERVER_URL,
        client_config=_get_client_config(),
    )
    msg = Message(
        message_id="nb-msg-001",
        role=Role.user,
        parts=[Part(root=TextPart(kind="text", text=question))],
    )
    async for item in client.send_message(msg):
        return _extract_text(item)
    return "(no response received)"


q = "What is the annual deductible for an individual?"
a = await ask(q)
print(f"Q: {q}")
print(f"A: {a}")

In [ ]:
# ── Step 3: More questions ────────────────────────────────────────

questions = [
    "What is the monthly premium for a family plan?",
    "Does the policy cover rental car reimbursement?",
    "How do I file a claim?",
]

for q in questions:
    a = await ask(q)
    print(f"Q: {q}")
    print(f"A: {a}")
    print()